### 練習問題2.43
Louis Reasonerは練習問題2.42を解くのにだいぶ苦労している。  
彼が書いたqueens⼿続きは動いてはいるようだが、  
⾮常に遅いのだ(6×6の場合でも、Louisは待ちきれなくなっ てしまう)。  
LouisがEva Lu Atorに⾒てくれるよう頼むと、  
彼⼥はLouisがflatmap内のマップのネストの順番を次のように逆にしてしまっていることを指摘した。 

    (flatmap (lambda (new-row)
                (map (lambda (rest-of-queens)
                            (adjoin-position new-row k rest-of-queens))
                     (queen-cols (- k 1)))
                     )
             (enumerate-interval 1 board-size)
             )  

逆にするとなぜプログラムの実⾏が遅くなるのか説明せよ。  
練習問題2.42のプログラムが8クイーンパズルを解く時間をTとして、   
Louisのプログラムがパズルを解くのにかかる時間を⾒積もれ。

Louisのプログラムの時間の見積もりの回答については割愛する。  

Louisのプログラムは、

- queen-cols手続き
- enumerate-interval手続き

の呼び出しが逆になっていることから、  
(enumerate-interval 1 board-size)で繰り返す箇所で  
queen-cols手続きを呼び出すことで無駄なループになっており、これが実行時間が遅くなることが伺える。  
Louisのプログラムは実行時間が遅いが、  
動作確認してみると、 

- 正しい組み合わせが出力される
- 重複した組み合わせが出力されることはない

ということから、  
プロセス図を記述してみて、どのような動作になっているかについては考えてみる価値はあるかと思われる。  
練習問題2.42のプログラムのプロセス図と、  
Louisのプログラムのプロセス図については、  
[練習問題2.42 8クイーンパズルのプロセス図](../exercises/2.43.xlsx)  
を参照。  
練習問題2.42のプロセス図は、  
バックトラックアルゴリズムにより、k<board-size(k:処理対象の列番号)のときに、  
safe?手続きにより、NGのものはそれ以降組み合わせを考えることがなく、無駄な試行がないことが分かる。  
それに対し、練習問題2.43(Louisのプログラム)のプロセス図は、  
k=board-sizeの動作は無駄がないが、  
k<board-sizeのときは1度チェックした組み合わせについても  
safe?手続きを再起呼び出しすることが何度も（指数関数的）に呼び出しているのが分かる。  
k<board-sizeのときは、同じ組み合わせを何度もチェックはするが、  
k=board-sizeまで含めると異なる組み合わせとなっているので重複した回答になることはない。  
k<board-sizeの場合は、  
全組み合わせを列挙しているわけではないが、  
それに近いステップ数の増加オーダーであることが考えられる。  

In [1]:
; 練習問題2.42の回答
; 問題文のqueens手続き
; 以下の定義が必要
; ・empty-board
; ・safe?手続き
; ・adjoin-position手続き
(define (queens board-size)
  (define (queen-cols k)
    (if (= k 0) (list empty-board)
        (filter
         (lambda (positions) (safe? k positions))
          (flatmap (lambda (rest-of-queens)
                    (map (lambda (new-row) (adjoin-position new-row k rest-of-queens))
                         (enumerate-interval 1 board-size))
                    )
                  (queen-cols (- k 1))
          )
         )
        )
    )
  (queen-cols board-size)
  )

(define (flatmap proc seq)
  (accumulate append '() (map proc seq)))

; 整数列の列挙
(define (enumerate-interval low high)
  (if (> low high) '()
      (cons low (enumerate-interval (+ low 1) high))))

; 集積
(define (accumulate op initial sequence)
  (if (null? sequence) initial
      (op (car sequence) (accumulate op initial (cdr sequence)))))

; フィルタリング
(define (filter predicate sequence)
  (cond ((null? sequence) '())
        ((predicate (car sequence))(cons (car sequence) (filter predicate (cdr sequence))))
        (else (filter predicate (cdr sequence))))
  )

; 回答1
(define empty-board '())

; 回答2
; adjoin-position手続きの呼び出し方が決まっているため、
; この回答しかないはず。
; 列ごとの位置(Y座標)をリストで表す。
(define (adjoin-position new-row k rest-of-queens)
  (append rest-of-queens (list new-row))
  )

; k列目の位置のy座標を返す
(define (get-k-y k positions)
  (define (iter count pos)
    (if (>= count k) (car pos)
        (iter (+ count 1) (cdr pos))
        )
    )
  (iter 1 positions)
  )

; 位置情報のダンプ
(define (print-pos positions)
  (if (null? positions) ()
      (begin
       (display (car positions))
       (display " ")
       (print-pos (cdr positions))
       )
      )
  )

; 回答3
(define (safe? k positions)
  (begin
   ;(display "'")
   ;(print-pos positions)
   (display positions)
   (newline)
   (let ((k-y-pos (get-k-y k positions)))
    (define (iter count pos)
      (if (>= count k) #t
        (if (null? pos) #t
            (let (
                  (y (car pos))
                  )
              (let ((y-upper (- y (- k count)))
                    (y-lower (+ y (- k count))))
                ; y座標が同じか、対角線上にあればFalseを返す。
                ; それ以外はチェックを継続する。
                (if (or (= k-y-pos y) (= k-y-pos y-upper) (= k-y-pos y-lower)) #f
                    (iter (+ count 1) (cdr pos))
                    )
                )
              )
            )
        )
      )
      (iter 1 positions)
    )
   )
  )

In [2]:
(queens 3)

(1)
(2)
(3)
(1 1)
(1 2)
(1 3)
(2 1)
(2 2)
(2 3)
(3 1)
(3 2)
(3 3)
(1 3 1)
(1 3 2)
(1 3 3)
(3 1 1)
(3 1 2)
(3 1 3)


()

In [3]:
(queens 4)

(1)
(2)
(3)
(4)
(1 1)
(1 2)
(1 3)
(1 4)
(2 1)
(2 2)
(2 3)
(2 4)
(3 1)
(3 2)
(3 3)
(3 4)
(4 1)
(4 2)
(4 3)
(4 4)
(1 3 1)
(1 3 2)
(1 3 3)
(1 3 4)
(1 4 1)
(1 4 2)
(1 4 3)
(1 4 4)
(2 4 1)
(2 4 2)
(2 4 3)
(2 4 4)
(3 1 1)
(3 1 2)
(3 1 3)
(3 1 4)
(4 1 1)
(4 1 2)
(4 1 3)
(4 1 4)
(4 2 1)
(4 2 2)
(4 2 3)
(4 2 4)
(1 4 2 1)
(1 4 2 2)
(1 4 2 3)
(1 4 2 4)
(2 4 1 1)
(2 4 1 2)
(2 4 1 3)
(2 4 1 4)
(3 1 4 1)
(3 1 4 2)
(3 1 4 3)
(3 1 4 4)
(4 1 3 1)
(4 1 3 2)
(4 1 3 3)
(4 1 3 4)


((2 4 1 3) (3 1 4 2))

In [4]:
; Louisのプログラム(間違った実装)
; ダンプ処理付き
(define (queens-ng board-size)
  (define (queen-cols k)
    (if (= k 0) (list empty-board)
        (filter
         (lambda (positions) (safe? k positions))
          ;(flatmap (lambda (rest-of-queens)
          ;          (map (lambda (new-row) (adjoin-position new-row k rest-of-queens))
          ;               (enumerate-interval 1 board-size))
          ;          )
          ;        (queen-cols (- k 1))
          ;)
         (flatmap (lambda (new-row)
                    (map (lambda (rest-of-queens)
                           (adjoin-position new-row k rest-of-queens))
                         (queen-cols (- k 1)))
                    )
                  (enumerate-interval 1 board-size)
                  )  
         )
        )
    )
  (queen-cols board-size)
  )


In [5]:
(queens-ng 3)

(1)
(2)
(3)
(1)
(2)
(3)
(1)
(2)
(3)
(1 1)
(2 1)
(3 1)
(1 2)
(2 2)
(3 2)
(1 3)
(2 3)
(3 3)
(1)
(2)
(3)
(1)
(2)
(3)
(1)
(2)
(3)
(1 1)
(2 1)
(3 1)
(1 2)
(2 2)
(3 2)
(1 3)
(2 3)
(3 3)
(1)
(2)
(3)
(1)
(2)
(3)
(1)
(2)
(3)
(1 1)
(2 1)
(3 1)
(1 2)
(2 2)
(3 2)
(1 3)
(2 3)
(3 3)
(3 1 1)
(1 3 1)
(3 1 2)
(1 3 2)
(3 1 3)
(1 3 3)


()

In [6]:
(queens-ng 4)

(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1 1)
(2 1)
(3 1)
(4 1)
(1 2)
(2 2)
(3 2)
(4 2)
(1 3)
(2 3)
(3 3)
(4 3)
(1 4)
(2 4)
(3 4)
(4 4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1 1)
(2 1)
(3 1)
(4 1)
(1 2)
(2 2)
(3 2)
(4 2)
(1 3)
(2 3)
(3 3)
(4 3)
(1 4)
(2 4)
(3 4)
(4 4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1 1)
(2 1)
(3 1)
(4 1)
(1 2)
(2 2)
(3 2)
(4 2)
(1 3)
(2 3)
(3 3)
(4 3)
(1 4)
(2 4)
(3 4)
(4 4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1 1)
(2 1)
(3 1)
(4 1)
(1 2)
(2 2)
(3 2)
(4 2)
(1 3)
(2 3)
(3 3)
(4 3)
(1 4)
(2 4)
(3 4)
(4 4)
(3 1 1)
(4 1 1)
(4 2 1)
(1 3 1)
(1 4 1)
(2 4 1)
(3 1 2)
(4 1 2)
(4 2 2)
(1 3 2)
(1 4 2)
(2 4 2)
(3 1 3)
(4 1 3)
(4 2 3)
(1 3 3)
(1 4 3)
(2 4 3)
(3 1 4)
(4 1 4)
(4 2 4)
(1 3 4)
(1 4 4)
(2 4 4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1 1)
(2 1)
(3 1)
(4 1)
(1 2)
(2 2)
(3 2)
(4 2)
(1 3)
(2 3)
(3 3)
(4 3)
(1 4)
(2 4)
(3 4)
(4 4)
(1)
(2)


((3 1 4 2) (2 4 1 3))